In [224]:
!pip install tensorflow-addons==0.16.1
import tensorflow_addons as tfa
import warnings
warnings.filterwarnings('ignore')

In [330]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn import metrics, preprocessing
from sklearn.preprocessing import LabelEncoder, StandardScaler

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils import clip_grad_norm_
from sklearn.preprocessing import StandardScaler

In [226]:
ratings = pd.read_csv('BX-Book-Ratings.csv', sep=";", on_bad_lines='warn', encoding="latin-1")

In [227]:
column_names = ratings.columns
print(column_names)

Index(['User-ID', 'ISBN', 'Book-Rating'], dtype='object')


In [228]:
books = pd.read_csv('BX-Books.csv', sep=';', encoding='latin-1', error_bad_lines=False)
print(books.columns)
if 'Image-URL-S' in books.columns and 'Image-URL-M' in books.columns and 'Image-URL-L' in books.columns:
    books = books.drop(columns=['Image-URL-S', 'Image-URL-M', 'Image-URL-L'])
print(books.head(5))

Skipping line 6452: expected 8 fields, saw 9
Skipping line 43667: expected 8 fields, saw 10
Skipping line 51751: expected 8 fields, saw 9

Skipping line 92038: expected 8 fields, saw 9
Skipping line 104319: expected 8 fields, saw 9
Skipping line 121768: expected 8 fields, saw 9

Skipping line 144058: expected 8 fields, saw 9
Skipping line 150789: expected 8 fields, saw 9
Skipping line 157128: expected 8 fields, saw 9
Skipping line 180189: expected 8 fields, saw 9
Skipping line 185738: expected 8 fields, saw 9

Skipping line 209388: expected 8 fields, saw 9
Skipping line 220626: expected 8 fields, saw 9
Skipping line 227933: expected 8 fields, saw 11
Skipping line 228957: expected 8 fields, saw 10
Skipping line 245933: expected 8 fields, saw 9
Skipping line 251296: expected 8 fields, saw 9
Skipping line 259941: expected 8 fields, saw 9
Skipping line 261529: expected 8 fields, saw 9



Index(['ISBN', 'Book-Title', 'Book-Author', 'Year-Of-Publication', 'Publisher',
       'Image-URL-S', 'Image-URL-M', 'Image-URL-L'],
      dtype='object')
         ISBN                                         Book-Title  \
0  0195153448                                Classical Mythology   
1  0002005018                                       Clara Callan   
2  0060973129                               Decision in Normandy   
3  0374157065  Flu: The Story of the Great Influenza Pandemic...   
4  0393045218                             The Mummies of Urumchi   

            Book-Author Year-Of-Publication                   Publisher  
0    Mark P. O. Morford                2002     Oxford University Press  
1  Richard Bruce Wright                2001       HarperFlamingo Canada  
2          Carlo D'Este                1991             HarperPerennial  
3      Gina Bari Kolata                1999        Farrar Straus Giroux  
4       E. J. W. Barber                1999  W. W. Norton &amp; Co

In [229]:
column_names = books.columns
print(column_names)

Index(['ISBN', 'Book-Title', 'Book-Author', 'Year-Of-Publication',
       'Publisher'],
      dtype='object')


In [230]:
users = pd.read_csv('BX-Users.csv', sep=";", on_bad_lines='warn', encoding="latin-1")

In [231]:
column_names = users.columns
print(column_names)

Index(['User-ID', 'Location', 'Age'], dtype='object')


In [315]:
combined = users.merge(ratings, on='User-ID').merge(books, on='ISBN')

lbl_isbn = preprocessing.LabelEncoder()
lbl_title = preprocessing.LabelEncoder()
lbl_author = preprocessing.LabelEncoder()
lbl_publisher = preprocessing.LabelEncoder()
lbl_location = preprocessing.LabelEncoder()

combined['ISBN'] = lbl_title.fit_transform(combined['ISBN'].values)
combined['Book-Title'] = lbl_title.fit_transform(combined['Book-Title'].values)
combined['Book-Author'] = lbl_author.fit_transform(combined['Book-Title'].values)
combined['Publisher'] = lbl_publisher.fit_transform(combined['Book-Title'].values)
combined['Year-Of-Publication'] = combined['Year-Of-Publication'].apply(lambda x: str(x).split('.')[0])
combined['Year-Of-Publication'] = combined['Year-Of-Publication'].replace('DK Publishing Inc', 0)
combined['Year-Of-Publication'] = combined['Year-Of-Publication'].replace('Gallimard', 0)
combined['Year-Of-Publication'] = combined['Year-Of-Publication'].astype(int)
combined['Age'] = combined['Age'].fillna(0).astype(int)
combined['Location'] = lbl_location.fit_transform(combined['Location'].values)
print(combined.head(5))

   User-ID  Location  Age   ISBN  Book-Rating  Book-Title  Book-Author  \
0        2     19186   18  24927            0       36197        36197   
1        8     19911    0     73            5       36036        36036   
2    11400     14894   49     73            0       36036        36036   
3    11676     13469    0     73            8       36036        36036   
4    41385     19330    0     73            0       36036        36036   

   Year-Of-Publication  Publisher  \
0                 2002      36197   
1                 2001      36036   
2                 2001      36036   
3                 2001      36036   
4                 2001      36036   

                                         Image-URL-S  \
0  http://images.amazon.com/images/P/0195153448.0...   
1  http://images.amazon.com/images/P/0002005018.0...   
2  http://images.amazon.com/images/P/0002005018.0...   
3  http://images.amazon.com/images/P/0002005018.0...   
4  http://images.amazon.com/images/P/0002005018.0...

In [316]:
'''to_drop = ['Image-URL-S', 'Image-URL-M', 'Image-URL-L']

data = data.drop(to_drop, axis=1, inplace=False)'''

"to_drop = ['Image-URL-S', 'Image-URL-M', 'Image-URL-L']\n\ndata = data.drop(to_drop, axis=1, inplace=False)"

In [317]:
class BookDataset(Dataset):
    def __init__(self, data) -> None:
        self.isbns = data['ISBN']
        self.book_titles = data['Book-Title']
        self.book_authors = data['Book-Author']
        self.year_of_publications = data['Year-Of-Publication']
        self.publishers = data['Publisher']
        self.user_ids = data['User-ID']
        self.locations = data['Location']
        self.ages = data['Age']
        self.book_ratings = data['Book-Rating']

    def __len__(self):
        return len(self.isbns)

    def __getitem__(self, item):
        isbn = self.isbns[item]
        book_title = self.book_titles[item]
        book_author = self.book_authors[item]
        year_of_publication = self.year_of_publications[item]
        publisher = self.publishers[item]
        user_id = self.user_ids[item]
        location = self.locations[item]
        age = self.ages[item]
        book_rating = self.book_ratings[item]

        return {
            'isbn': torch.tensor(isbn, dtype=torch.long),
            'book_title': torch.tensor(book_title, dtype=torch.long),
            'book_author': torch.tensor(book_author, dtype=torch.long),
            'publisher': torch.tensor(publisher, dtype=torch.long),
            'user_id': torch.tensor(user_id, dtype=torch.long),
            'location': torch.tensor(location, dtype=torch.long),
            'year_of_publication': torch.tensor(year_of_publication, dtype=torch.float),
            'age': torch.tensor(age, dtype=torch.float),
            'book_rating': torch.tensor(book_rating, dtype=torch.float),
        }

In [319]:
dataset = BookDataset(combined)

In [320]:
data

<function __main__.data(ratings, books, users)>

In [321]:
combined

,User-ID,Location,Age,ISBN,Book-Rating,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
0,2,19186,18,24927,0,36197,36197,2002,36197,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...
1,8,19911,0,73,5,36036,36036,2001,36036,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...
2,11400,14894,49,73,0,36036,36036,2001,36036,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...
3,11676,13469,0,73,8,36036,36036,2001,36036,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...
4,41385,19330,0,73,0,36036,36036,2001,36036,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...
...,...,...,...,...,...,...,...,...,...,...,...,...
1031131,278851,4986,33,156891,0,16706,16706,2000,16706,http://images.amazon.com/images/P/0743203763.0...,http://images.amazon.com/images/P/0743203763.0...,http://images.amazon.com/images/P/0743203763.0...
1031132,278851,4986,33,165573,5,11163,11163,2001,11163,http://images.amazon.com/images/P/0767907566.0...,http://images.amazon.com/images/P/0767907566.0...,http://images.amazon.com/images/P/0767907566.0...
1031133,278851,4986,33,201536,7,234649,234649,1985,234649,http://images.amazon.com/images/P/0884159221.0...,http://images.amazon.com/images/P/0884159221.0...,http://images.amazon.com/images/P/0884159221.0...
1031134,278851,4986,33,207840,7,175773,175773,1997,175773,http://images.amazon.com/images/P/0912333022.0...,http://images.amazon.com/images/P/0912333022.0...,http://images.amazon.com/images/P/0912333022.0...


In [322]:
dataset

In [325]:
data = pd.merge(ratings, users, on='User-ID', how='inner')

In [326]:
data = pd.merge(data, books, on='ISBN', how='inner')

In [329]:
year = pd.to_numeric(data['Year-Of-Publication'], errors='coerce').fillna(2099, downcast='infer')


In [327]:
data.columns

Index(['User-ID', 'ISBN', 'Book-Rating', 'Location', 'Age', 'Book-Title',
       'Book-Author', 'Year-Of-Publication', 'Publisher', 'Image-URL-S',
       'Image-URL-M', 'Image-URL-L'],
      dtype='object')

In [323]:
class WideAndDeepModel(nn.Module):
    def __init__(self, embedding_sizes, n_continuous):
        super(WideAndDeepModel, self).__init__()
        self.embeddings = nn.ModuleList([nn.Embedding(categories, size) for categories, size in embedding_sizes])
        n_emb = sum(e.embedding_dim for e in self.embeddings)  # sum of all embeddings dimensions
        self.n_emb, self.n_continuous = n_emb, n_continuous
        self.lin1 = nn.Linear(self.n_emb + self.n_continuous, 200)
        self.lin2 = nn.Linear(200, 70)
        self.lin3 = nn.Linear(70, 11)  # 11 output classes for Book-Rating
        self.bn1 = nn.BatchNorm1d(self.n_continuous)
        self.bn2 = nn.BatchNorm1d(200)
        self.bn3 = nn.BatchNorm1d(70)
        self.emb_drop = nn.Dropout(0.6)
        self.drops = nn.Dropout(0.3)

    def forward(self, x_cat, x_continuous):
        x = [e(x_cat[:, i]) for i, e in enumerate(self.embeddings)]
        x = torch.cat(x, 1)
        x = self.emb_drop(x)

        x_continuous = self.bn1(x_continuous)
        x = torch.cat([x, x_continuous], 1)

        x = F.relu(self.lin1(x))
        x = self.drops(x)
        x = self.bn2(x)
        x = F.relu(self.lin2(x))
        x = self.drops(x)
        x = self.bn3(x)
        x = self.lin3(x)

        return x

# Training function
def train_model(model, criterion, optimizer, cat_train, cont_train, y_train, cat_test, cont_test, y_test, n_epochs):
    for epoch in range(n_epochs):
        model.train()
        optimizer.zero_grad()

        y_pred = model(cat_train, cont_train)
        loss = criterion(y_pred, y_train)
        loss.backward()
        optimizer.step()

        model.eval()
        with torch.no_grad():
            y_val = model(cat_test, cont_test)
            val_loss = criterion(y_val, y_test)

        print(f"Epoch {epoch+1}/{n_epochs} - Loss: {loss.item()}, Val Loss: {val_loss.item()}")

# Split the data
train_data, test_data = train_test_split(dataset, test_size=0.2, random_state=2023)

# Define model parameters
embedding_sizes = [(len(label_encoders[col].classes_), min(50, (len(label_encoders[col].classes_)+1)//2)) for col in ['ISBN', 'Book-Title', 'Book-Author', 'Year-Of-Publication', 'Publisher', 'Location']]
continuous_feature_age = 1

model = WideAndDeepModel(embedding_sizes, continuous_feature_age)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

# Convert data to tensors
cat_train = torch.tensor(train_data[['ISBN', 'Book-Title', 'Book-Author', 'Year-Of-Publication', 'Publisher', 'Location']].values, dtype=torch.int64)
cont_train = torch.tensor(train_data[['Age']].values, dtype=torch.float32)
y_train = torch.tensor(train_data[['Book-Rating']].values, dtype=torch.int64).squeeze()

cat_test = torch.tensor(test_data[['ISBN', 'Book-Title', 'Book-Author', 'Year-Of-Publication', 'Publisher', 'Location']].values, dtype=torch.int64)
cont_test = torch.tensor(test_data[['Age']].values, dtype=torch.float32)
y_test = torch.tensor(test_data[['Book-Rating']].values, dtype=torch.int64).squeeze()

# Train the model
n_epochs = 10
train_model(model, criterion, optimizer, cat_train, cont_train, y_train, cat_test, cont_test, y_test, n_epochs)

# Use the model for predictions
with torch.no_grad():
    final_preds = model(cat_test, cont_test)
    final_preds = torch.argmax(final_preds, dim=1)
    accuracy = (final_preds == y_test).float().mean()

print(f"Prediction Accuracy: {accuracy*100:.2f}%")

Skipping line 6452: expected 8 fields, saw 9
Skipping line 43667: expected 8 fields, saw 10
Skipping line 51751: expected 8 fields, saw 9

Skipping line 92038: expected 8 fields, saw 9
Skipping line 104319: expected 8 fields, saw 9
Skipping line 121768: expected 8 fields, saw 9

Skipping line 144058: expected 8 fields, saw 9
Skipping line 150789: expected 8 fields, saw 9
Skipping line 157128: expected 8 fields, saw 9
Skipping line 180189: expected 8 fields, saw 9
Skipping line 185738: expected 8 fields, saw 9

Skipping line 209388: expected 8 fields, saw 9
Skipping line 220626: expected 8 fields, saw 9
Skipping line 227933: expected 8 fields, saw 11
Skipping line 228957: expected 8 fields, saw 10
Skipping line 245933: expected 8 fields, saw 9
Skipping line 251296: expected 8 fields, saw 9
Skipping line 259941: expected 8 fields, saw 9
Skipping line 261529: expected 8 fields, saw 9



Epoch 1/10 - Loss: 2.4820892810821533, Val Loss: 2.2997348308563232
Epoch 2/10 - Loss: 2.3714795112609863, Val Loss: 2.257392406463623
Epoch 3/10 - Loss: 2.3169758319854736, Val Loss: 2.2171759605407715
Epoch 4/10 - Loss: 2.2675535678863525, Val Loss: 2.1843550205230713
Epoch 5/10 - Loss: 2.216362476348877, Val Loss: 2.1444199085235596
Epoch 6/10 - Loss: 2.1584932804107666, Val Loss: 2.081434488296509
Epoch 7/10 - Loss: 2.096243143081665, Val Loss: 2.0002758502960205
Epoch 8/10 - Loss: 2.0276741981506348, Val Loss: 1.9149459600448608
Epoch 9/10 - Loss: 1.9527101516723633, Val Loss: 1.8010380268096924
Epoch 10/10 - Loss: 1.876216173171997, Val Loss: 1.7233561277389526
Prediction Accuracy: 61.08%
